In [0]:
spark.conf.set(
  "spark.sql.streaming.stateStore.providerClass",
  "com.databricks.sql.streaming.state.RocksDBStateStoreProvider")

spark.conf.set(
  "spark.sql.streaming.stateStore.rocksdb.changelogCheckpointing.enabled", "true")

// 16 cores on my Databricks Cluster, 
// 3 slots for kafka (Because the Kafka topic has 3 partitions configured and Spark by default is 1:1), 
// 13 for stage 1 (the only stage), 
spark.conf.set("spark.sql.shuffle.partitions", "13")

In [0]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.streaming._

import org.apache.spark.sql.Encoder
import org.apache.spark.sql.Encoders

// All of the TransformWithState code and tests are in here
import demo.heat._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.Encoder
import org.apache.spark.sql.Encoders
import demo.heat._

In [0]:
val kafkaBootstrap = "pkc-56d1g.eastus.azure.confluent.cloud:9092"
val eventsTopic    = "player_signals"

val kafkaBootstrap: String = pkc-56d1g.eastus.azure.confluent.cloud:9092
val eventsTopic: String = player_signals

In [0]:
val itemSchema = new StructType()
  .add("slot", IntegerType)
  .add("itemID", LongType)
  .add("displayName", StringType)
  .add("totalCost", IntegerType)   // from DDragon enrichment
  .add("count", IntegerType)

val scoresSchema = new StructType()
  .add("kills", IntegerType)
  .add("deaths", IntegerType)
  .add("assists", IntegerType)
  .add("creepScore", IntegerType)

val payloadSchema = new StructType()
  // snapshot fields
  .add("level", IntegerType)
  .add("items", ArrayType(itemSchema))
  .add("scores", scoresSchema)
  // pulse fields
  .add("etype", StringType)        // ChampionKill, DragonKill, ...
  .add("role", StringType)         // killer | victim | assister
  .add("eventId", LongType)
  .add("timeSec", DoubleType)
  .add("dragonType", StringType)
  .add("stolen", BooleanType)
  .add("turret", StringType)
  .add("firstBlood", BooleanType)
  .add("championName", StringType)
  .add("isDead", BooleanType)

// ---- top-level envelope ----
val signalSchema = new StructType()
  .add("kind", StringType)         // "snapshot" | "pulse"
  .add("tsMillis", LongType)
  .add("gameId", StringType)
  .add("riotId", StringType)
  .add("team", StringType)
  .add("payload", payloadSchema)

val itemSchema: org.apache.spark.sql.types.StructType = StructType(StructField(itemID,LongType,true),StructField(displayName,StringType,true),StructField(totalCost,IntegerType,true),StructField(count,IntegerType,true),StructField(slot,IntegerType,true))
val scoresSchema: org.apache.spark.sql.types.StructType = StructType(StructField(kills,IntegerType,true),StructField(deaths,IntegerType,true),StructField(assists,IntegerType,true),StructField(creepScore,IntegerType,true))
val payloadSchema: org.apache.spark.sql.types.StructType = StructType(StructField(level,IntegerType,true),StructField(items,ArrayType(StructType(StructField(itemID,LongType,true),StructField(displayName,StringType,true),StructField(totalCost,IntegerType,true),StructField(count,IntegerType,true),StructField(slot,IntegerType,true)),true),true),StructField(scores,StructType(StructField(kills,IntegerType,true),StructField(deaths,IntegerType,true),StructField(assists,IntegerType,true),StructField(creepScore,IntegerType,true)),true),StructField(etype,StringType,true),StructField(role,StringType,true),StructField(eventId,LongType,true),StructField(timeSec,DoubleType,true),StructField(dragonType,StringType,true),StructField(stolen,BooleanType,true),StructField(turret,StringType,true),StructField(firstBlood,BooleanType,true),StructField(championName,StringType,true))
val signalSchema: org.apache.spark.sql.types.StructType = StructType(StructField(kind,StringType,true),StructField(tsMillis,LongType,true),StructField(gameId,StringType,true),StructField(riotId,StringType,true),StructField(payload,StructType(StructField(level,IntegerType,true),StructField(items,ArrayType(StructType(StructField(itemID,LongType,true),StructField(displayName,StringType,true),StructField(totalCost,IntegerType,true),StructField(count,IntegerType,true),StructField(slot,IntegerType,true)),true),true),StructField(scores,StructType(StructField(kills,IntegerType,true),StructField(deaths,IntegerType,true),StructField(assists,IntegerType,true),StructField(creepScore,IntegerType,true)),true),StructField(etype,StringType,true),StructField(role,StringType,true),StructField(eventId,LongType,true),StructField(timeSec,DoubleType,true),StructField(dragonType,StringType,true),StructField(stolen,BooleanType,true),StructField(turret,StringType,true),StructField(firstBlood,BooleanType,true),StructField(championName,StringType,true)),true))

In [0]:
// This is useful for testing so I don't have to actually run the game
// or worry about what's on the topic
def getStreamingDF(useDelta: Boolean = false): DataFrame = {
  useDelta match {
    case true => 
      spark.readStream.table("users.alex_vanadio.player_signals")
    case false =>
      spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", kafkaBootstrap)
        .option("startingOffsets", "latest")
        .option("kafka.security.protocol", "SASL_SSL")
        .option("kafka.sasl.mechanism", "PLAIN")
        .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='DUZSLDUERISLSVET' password='cfltiL+DtOkmn4axL6BPsTr5+N//BAt7lQECDGcUdVpvI6HrVx75ICzlWbxs2h5g';")
        .option("subscribe", eventsTopic)
        .load()
  }
}

def getStreamingDF(useDelta: Boolean): org.apache.spark.sql.DataFrame

In [0]:
val playerSignalsStreamingDF = getStreamingDF(useDelta = false)

val playerSignals = playerSignalsStreamingDF
  .selectExpr("topic", "partition", "offset", "CAST(key AS STRING) as key", "CAST(value AS STRING) AS json", "timestamp AS kafkaTs")
  .select(col("*"), from_json(col("json"), signalSchema).as("s"), col("kafkaTs"))
  .select(
    col("key").as("kafkaKey"),
    col("kafkaTs"),
    col("s.kind"),
    col("s.tsMillis"),
    upper(trim(col("s.gameId"))).as("gameId"),
    trim(col("s.riotId")).as("riotId"),
    upper(trim(col("s.team"))).as("team"), 
    col("s.payload.*"),   // expands union payload fields; unrelated ones will be null
    col("topic"),
    col("partition"),
    col("offset")
  )

// display(playerSignals)

val playerSignalsStreamingDF: org.apache.spark.sql.DataFrame = [key: binary, value: binary ... 5 more fields]
val playerSignals: org.apache.spark.sql.DataFrame = [kafkaKey: string, kafkaTs: timestamp ... 20 more fields]

In [ ]:
val playerSignalsWithItems = playerSignals
  .withColumn(
    "items_str",
    when(col("kind") === "snapshot" && col("items").isNotNull,
      array_join(
        transform(
          array_sort(
            transform(col("items"),
              x => struct(
                x.getField("slot").as("slot"),
                x.getField("itemID").as("itemID"),
                x.getField("displayName").as("displayName")
              )
            )
          ),
          x => format_string(
            "%s:%s:%s",
            x.getField("slot"),
            x.getField("itemID"),
            coalesce(x.getField("displayName"), lit(""))
          )
        ),
        ", "
      )
    ).otherwise(lit(null).cast("string")) // => NULL for non-snapshots (and when items is NULL)
  )

// display(playerSignalsWithItems)

This is a neat optimization because we know how small the item table really is. LoL has < 1000 items.

The other approach is to do a static-stream join, but this should be much more performant

In [0]:
// Build driver-side lookup once
val costMapLocal: Map[Long, Double] =
  spark.table("users.alex_vanadio.ddragon_items")
    .select($"itemId".cast(LongType), $"totalCost".cast(DoubleType))
    .collect()
    .map(r => r.getLong(0) -> r.getDouble(1)).toMap

val costMapCol = typedLit(costMapLocal) // MapType(LongType, DoubleType)

// Compute invValue only on snapshots (no joins)
val playerSignalsWithInv = playerSignalsWithItems
  .withColumn(
    "invValue",
    when($"kind" === "snapshot",
      aggregate(
        coalesce($"items", array()),           // items: array<struct<itemID, count, ...>>
        lit(0.0),                              // zero init
        (acc, it) =>
          acc +
            coalesce(                            // price * count
              element_at(costMapCol, it.getField("itemID").cast(LongType)),
              lit(0.0)
            ) * coalesce(it.getField("count").cast(IntegerType), lit(1))
      )
    ).otherwise(lit(null).cast(DoubleType))
  )
  .drop("items")

// display(playerSignalsWithInv)

val costMapLocal: Map[Long,Double] = HashMap(2163 -> 1000.0, 223047 -> 500.0, 1036 -> 350.0, 223084 -> 2500.0, 9273 -> 0.0, 2031 -> 150.0, 9180 -> 0.0, 1031 -> 800.0, 3863 -> 400.0, 1083 -> 450.0, 4015 -> 2500.0, 6690 -> 775.0, 9305 -> 0.0, 1501 -> 0.0, 9405 -> 0.0, 3053 -> 3200.0, 447118 -> 1000.0, 4005 -> 2250.0, 3399 -> 0.0, 226632 -> 2500.0, 1105 -> 450.0, 447107 -> 1000.0, 446656 -> 1000.0, 3172 -> 1600.0, 663056 -> 2500.0, 226333 -> 2500.0, 3858 -> 400.0, 228008 -> 9000.0, 3012 -> 900.0, 443063 -> 1000.0, 226691 -> 2500.0, 9400 -> 0.0, 323004 -> 2900.0, 3072 -> 3400.0, 220011 -> 750.0, 1522 -> 0.0, 6695 -> 2500.0, 3040 -> 2900.0, 3153 -> 3200.0, 226664 -> 2500.0, 446693 -> 1000.0, 3108 -> 850.0, 9184 -> 0.0, 220001 -> 2000.0, 443069 -> 1000.0, 6665 -> 3200.0, 3175 -> 1600.0, 2141 -> 300.0, 223072 -> 2500.0, 3121 -> 2400.0, 226667 -> 2500.0, 9278 -> 0.0, 2146 -> 0.0, 4633 -> 3100.0, 223116 -> 2500.0, 4403 -> 7187.0, 3004 -> 2900.0, 6675 -> 2650.0, 3135 -> 3000.0, 4011 -> 2500.0, 3044 -> 1100.0, 223094 -> 2500.0, 6655 -> 2750.0, 4638 -> 1100.0, 223748 -> 2500.0, 223003 -> 2500.0, 1507 -> 0.0, 1001 -> 300.0, 2010 -> 50.0, 223111 -> 500.0, 3140 -> 1300.0, 1518 -> 0.0, 223185 -> 500.0, 9292 -> 0.0, 443058 -> 1000.0, 223067 -> 500.0, 223143 -> 2500.0, 2020 -> 1337.0, 3513 -> 0.0, 4637 -> 3000.0, 1055 -> 450.0, 9300 -> 0.0, 220000 -> 750.0, 3131 -> 2300.0, 3116 -> 2600.0, 3046 -> 2650.0, 223105 -> 500.0, 2501 -> 3300.0, 220006 -> 2000.0, 6676 -> 3000.0, 223814 -> 2500.0, 322065 -> 2600.0, 3089 -> 3500.0, 6662 -> 2900.0, 226671 -> 2500.0, 3803 -> 1300.0, 3855 -> 400.0, 323222 -> 2800.0, 9172 -> 0.0, 9287 -> 0.0, 1511 -> 0.0, 663193 -> 2500.0, 2420 -> 1600.0, 2049 -> 500.0, 223071 -> 2500.0, 667109 -> 2500.0, 3084 -> 3000.0, 223073 -> 2500.0, 3902 -> 0.0, 443081 -> 1000.0, 3866 -> 400.0, 444636 -> 1000.0, 2403 -> 0.0, 223020 -> 500.0, 4632 -> 1600.0, 444637 -> 1000.0, 226699 -> 2500.0, 9304 -> 0.0, 223137 -> 2500.0, 9283 -> 0.0, 3075 -> 2450.0, 447114 -> 1000.0, 220002 -> 2000.0, 223135 -> 2500.0, 3152 -> 2650.0, 224644 -> 2500.0, 3851 -> 400.0, 443056 -> 1000.0, 9401 -> 0.0, 6670 -> 1300.0, 223039 -> 2500.0, 1040 -> 350.0, 6657 -> 2600.0, 447111 -> 2500.0, 226665 -> 2500.0, 323003 -> 2900.0, 3068 -> 2700.0, 443059 -> 1000.0, 223157 -> 2500.0, 447101 -> 1000.0, 223177 -> 500.0, 3157 -> 3250.0, 6702 -> 0.0, 226695 -> 2500.0, 4643 -> 2300.0, 1104 -> 0.0, 3184 -> 950.0, 2508 -> 900.0, 223742 -> 2500.0, 6631 -> 3300.0, 8001 -> 2500.0, 3036 -> 3100.0, 663059 -> 2500.0, 9187 -> 0.0, 6700 -> 2675.0, 323190 -> 2600.0, 6616 -> 2250.0, 3870 -> 400.0, 3211 -> 1250.0, 1521 -> 0.0, 9272 -> 0.0, 4628 -> 2750.0, 222502 -> 2500.0, 226673 -> 2500.0, 3011 -> 1900.0, 3857 -> 400.0, 224637 -> 2500.0, 4629 -> 3000.0, 446691 -> 1000.0, 221011 -> 500.0, 223156 -> 2500.0, 447100 -> 1000.0, 447108 -> 1000.0, 1057 -> 850.0, 226692 -> 2500.0, 223504 -> 2500.0, 9308 -> 0.0, 223002 -> 2500.0, 323050 -> 2300.0, 9175 -> 0.0, 223095 -> 2500.0, 443062 -> 1000.0, 4016 -> 2500.0, 1042 -> 250.0, 1502 -> 0.0, 3742 -> 2900.0, 223152 -> 2500.0, 2150 -> 0.0, 3814 -> 3000.0, 6696 -> 2750.0, 1519 -> 0.0, 3867 -> 400.0, 222503 -> 2500.0, 220010 -> 750.0, 663039 -> 2500.0, 3134 -> 1000.0, 223115 -> 2500.0, 2015 -> 700.0, 6620 -> 2200.0, 9177 -> 0.0, 223006 -> 500.0, 9190 -> 0.0, 9277 -> 0.0, 3877 -> 400.0, 3600 -> 0.0, 9301 -> 0.0, 3144 -> 600.0, 4402 -> 2950.0, 1027 -> 300.0, 3050 -> 2200.0, 226653 -> 2500.0, 3033 -> 3300.0, 1106 -> 450.0, 3137 -> 3000.0, 3065 -> 2700.0, 3349 -> 0.0, 226621 -> 2500.0, 223031 -> 2500.0, 6656 -> 2865.0, 3105 -> 1100.0, 223165 -> 2500.0, 6677 -> 1100.0, 3862 -> 400.0, 6691 -> 2950.0, 224645 -> 2500.0, 4636 -> 2765.0, 3102 -> 3000.0, 1006 -> 300.0, 221043 -> 500.0, 3119 -> 2400.0, 3078 -> 3333.0, 3916 -> 800.0, 3001 -> 2300.0, 4012 -> 3000.0, 226617 -> 2500.0, 3176 -> 1400.0, 3087 -> 2700.0, 228020 -> 2500.0, 3143 -> 2700.0, 3179 -> 2500.0, 223009 -> 500.0, 3094 -> 2650.0, 3010 -> 900.0, 664644 -> 2500.0, 9288 -> 0.0, 223026 -> 2500.0, 1039 -

In [0]:
import demo.heat._
implicit val heatInEnc : Encoder[HeatIn]  = Encoders.product[HeatIn]
implicit val heatOutEnc: Encoder[HeatOut] = Encoders.product[HeatOut]

// Keep just what we need, and skip rows without a player identity
val heatInDS =
  playerSignalsWithInv
    .filter($"gameId".isNotNull && $"riotId".isNotNull && $"kind".isNotNull && $"tsMillis".isNotNull && $"team".isNotNull)
    .select(
      $"gameId", $"team", $"riotId", $"tsMillis", $"kind", $"championName", $"isDead",
      $"etype", $"role",
      $"level", $"invValue", $"items_str",
      $"eventId", 
      $"scores.kills", $"scores.deaths", $"scores.assists", $"scores.creepScore"
    )
    .as[HeatIn]

// display(heatInDS)

import demo.heat._
val heatInEnc: org.apache.spark.sql.Encoder[demo.heat.HeatIn] = ProductEncoder(demo.heat.HeatIn,List(EncoderField(gameId,StringEncoder,true,{},None,None), EncoderField(team,StringEncoder,true,{},None,None), EncoderField(riotId,StringEncoder,true,{},None,None), EncoderField(tsMillis,PrimitiveLongEncoder,false,{},None,None), EncoderField(kind,StringEncoder,true,{},None,None), EncoderField(etype,OptionEncoder(StringEncoder),true,{},None,None), EncoderField(championName,OptionEncoder(StringEncoder),true,{},None,None), EncoderField(role,OptionEncoder(StringEncoder),true,{},None,None), EncoderField(level,OptionEncoder(PrimitiveIntEncoder),true,{},None,None), EncoderField(invValue,OptionEncoder(PrimitiveDoubleEncoder),true,{},None,None), EncoderField(eventId,OptionEncoder(PrimitiveLongEncoder),true,{},None,None), EncoderField(kills,OptionEncoder(PrimitiveIntEncoder),true,{},None,None), EncoderField(deaths,OptionEncoder(PrimitiveIntEncoder),true,{},None,None), EncoderField(assists,OptionEncoder(PrimitiveIntEncoder),true,{},None,None), EncoderField(creepScore,OptionEncoder(PrimitiveDoubleEncoder),true,{},None,None)),None)
val heatOutEnc: org.apache.spark.sql.Encoder[demo.heat.HeatOut] = ProductEncoder(demo.heat.HeatOut,List(EncoderField(gameId,StringEncoder,true,{},None,None), EncoderField(team,StringEncoder,true,{},None,None), EncoderField(riotId,StringEncoder,true,{},None,None), EncoderField(championName,StringEncoder,true,{},None,None), EncoderField(emitTsMs,PrimitiveLongEncoder,false,{},None,None), EncoderField(powerRaw,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(momentumRaw,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(powerNorm,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(momentumNorm,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(heat,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(spreeStreak,PrimitiveIntEncoder,false,{},None,None), EncoderField(kills,PrimitiveIntEncoder,false,{},None,None), EncoderField(deaths,PrimitiveIntEncoder,false,{},None,None), EncoderField(assists,PrimitiveIntEncoder,false,{},None,None), EncoderField(creepScore,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(invValue,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(level,PrimitiveIntEncoder,false,{},None,None), EncoderField(halfLifeSec,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(momentumCap,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(heatWPower,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(heatWMomentum,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(itemBudget,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(levelMax,PrimitiveDoubleEncoder,false,{},None,None), EncoderField(sourceTsMs,PrimitiveLongEncoder,false,{},None,None), EncoderField(backendLatencyMs,PrimitiveLongEncoder,false,{},None,None)),None)
val heatInDS: org.apache.spark.sql.Dataset[demo.heat.HeatIn] = [gameId: string, team: string ... 13 more fields]

In [0]:
import demo.heat._

val heatDS =
  heatInDS
    .groupByKey(h => s"${h.gameId}|${h.riotId}|${h.team}")
    .transformWithState(new HeatProcessor(TTLConfig(java.time.Duration.ofHours(2))),
                        TimeMode.ProcessingTime,
                        OutputMode.Update)

// display(heatDS)

import demo.heat._
val heatDS: org.apache.spark.sql.Dataset[demo.heat.HeatOut] = [gameId: string, team: string ... 23 more fields]

In [0]:
import org.apache.spark.sql.execution.streaming.RealTimeTrigger
import java.nio.file.Files

val uuid = java.util.UUID.randomUUID.toString
val cols = heatDS.columns.map(col).toIndexedSeq
val toKafka =
  heatDS.select(
    concat_ws("|", $"gameId", $"riotId", $"team").as("key"),
    to_json(struct(cols: _*)).as("value")
  )

toKafka
  .writeStream
  .format("kafka")
  .option("kafka.bootstrap.servers", kafkaBootstrap)
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='DUZSLDUERISLSVET' password='cfltiL+DtOkmn4axL6BPsTr5+N//BAt7lQECDGcUdVpvI6HrVx75ICzlWbxs2h5g';")
  .option("topic", "heat")
  .option("checkpointLocation", s"/tmp/alexv/$uuid")
  .trigger(RealTimeTrigger.apply())
  .outputMode(OutputMode.Update())
  .start()


import org.apache.spark.sql.execution.streaming.RealTimeTrigger
import java.nio.file.Files
val cols: IndexedSeq[org.apache.spark.sql.Column] = ArraySeq(gameId, team, riotId, championName, emitTsMs, powerRaw, momentumRaw, powerNorm, momentumNorm, heat, spreeStreak, kills, deaths, assists, creepScore, invValue, level, halfLifeSec, momentumCap, heatWPower, heatWMomentum, itemBudget, levelMax, sourceTsMs, backendLatencyMs)
val toKafka: org.apache.spark.sql.DataFrame = [key: string, value: string]
val res35: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6e30b734